In [ ]:
#!wget --header="Host: storage.googleapis.com" --header="User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9" --header="Accept-Language: en-US,en;q=0.9" --header="Referer: https://www.kaggle.com/" "https://storage.googleapis.com/kaggle-competitions-data/kaggle-v2/19018/1053282/bundle/archive.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1626438548&Signature=T8PmmW6ew4RlHDArZmh57HlsWRZu3m9r5DI7YimrU7poEc%2Bf2IoAzmTaIK%2FbMFdz6ndueG5AQLP%2BZ717fPTengiEK9657TymXNRE9pTHBPiLE2eU7vOewgfBcgluUbiJqAsk1CxlAchn4e0xXjhCXx1xO2wOyvQZs9%2FWPyuZNfC686EQybsbXs3WhnyaR4sHMl0sbTPHvEwQ2sxjF0L0zMMrw5VZ7%2BjnjCo%2BXrUNXq2h9FtorC%2BZ8Rzhc62tDZDf5iG6gxbqEiV03qR1C9aso0%2B5fDh1OJ8JA2bNBFXDteSMOrJ%2Fbhzx2vxlkPGcmMu5xk9yKnI9YwxZxCtOtDfyvg%3D%3D&response-content-disposition=attachment%3B+filename%3Djigsaw-multilingual-toxic-comment-classification.zip" -c -O 'jigsaw-multilingual-toxic-comment-classification.zip'

In [1]:
import numpy as np
import pandas as pd
import zipfile
import os

In [2]:
from transformers import TFXLMRobertaModel,XLMRobertaTokenizer

In [3]:
df1=pd.read_csv('../input/jigsaw-multilingual-toxic-comment-classification/jigsaw-toxic-comment-train.csv')

In [4]:
df1.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [5]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 223549 entries, 0 to 223548
Data columns (total 8 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   id             223549 non-null  object
 1   comment_text   223549 non-null  object
 2   toxic          223549 non-null  int64 
 3   severe_toxic   223549 non-null  int64 
 4   obscene        223549 non-null  int64 
 5   threat         223549 non-null  int64 
 6   insult         223549 non-null  int64 
 7   identity_hate  223549 non-null  int64 
dtypes: int64(6), object(2)
memory usage: 13.6+ MB


In [6]:
temp=np.where(np.sum(df1.loc[:,['toxic'	,'severe_toxic'	,'obscene'	,'threat'	,'insult'	,'identity_hate']].values,axis=1)>0,'toxic','non-toxic')

In [7]:
pd.Series(temp).value_counts()

non-toxic    201081
toxic         22468
dtype: int64

In [8]:
df2=pd.read_csv('../input/jigsaw-multilingual-toxic-comment-classification/jigsaw-unintended-bias-train.csv')

In [9]:
df2.head()

,id,comment_text,toxic,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,...,article_id,rating,funny,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count
0,59848,"This is so cool. It's like, 'would you want yo...",0.000000,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.0,0,4
1,59849,Thank you!! This would make my life a lot less...,0.000000,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.0,0,4
2,59852,This is such an urgent design problem; kudos t...,0.000000,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.0,0,4
3,59855,Is this something I'll be able to install on m...,0.000000,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.0,0,4
4,59856,haha you guys are a bunch of losers.,0.893617,0.021277,0.0,0.021277,0.87234,0.0,0.0,0.0,...,2006,rejected,0,0,0,1,0,0.0,4,47


In [10]:
df2.shape

(1902194, 45)

In [11]:
temp=np.where(np.sum(df2.loc[:,['toxic'	,'severe_toxicity'	,'obscene'	,'threat'	,'insult'	,'identity_attack']].values,axis=1)>0,'toxic','non-toxic')

In [12]:
pd.Series(temp).value_counts()

non-toxic    1311900
toxic         590294
dtype: int64

In [13]:
drop_columns=list(col for col in list(df2.columns) if not col in ['comment_text','toxic','severe_toxicity','obscene','threat','insult','identity_attack'])

In [14]:
df2.drop(drop_columns,axis=1,inplace=True)

In [15]:
df1.columns

Index(['id', 'comment_text', 'toxic', 'severe_toxic', 'obscene', 'threat',
       'insult', 'identity_hate'],
      dtype='object')

In [16]:
df1.drop('id',axis=1,inplace=True)

In [17]:
df1['target']=df1.apply(lambda x: 1 if (int(x['toxic'])+int(x['severe_toxic'])+int(x['obscene'])+int(x['threat'])+int(x['insult'])+int(x['identity_hate']))>0 else 0,axis=1)
df2['target']=df2.apply(lambda x: 1 if (float(x['toxic'])+float(x['severe_toxicity'])+float(x['obscene'])+float(x['threat'])+float(x['insult'])+float(x['identity_attack']))>0 else 0,axis=1)
df1.drop(['toxic'	,'severe_toxic'	,'obscene'	,'threat'	,'insult'	,'identity_hate'],axis=1,inplace=True)
df2.drop(['toxic'	,'severe_toxicity'	,'obscene'	,'threat'	,'insult'	,'identity_attack'],axis=1,inplace=True)

In [18]:
df1.head()

,comment_text,target
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [19]:
df2.head()

,comment_text,target
0,"This is so cool. It's like, 'would you want yo...",0
1,Thank you!! This would make my life a lot less...,0
2,This is such an urgent design problem; kudos t...,0
3,Is this something I'll be able to install on m...,0
4,haha you guys are a bunch of losers.,1


In [20]:
df1.shape[0]+df2.shape[0]

2125743

In [21]:
df=pd.concat([df1,df2],axis=0,ignore_index=True)

In [22]:
df.head()

,comment_text,target
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [23]:
df.reset_index(drop=True,inplace=True)

In [24]:
from tqdm import tqdm

In [25]:
df['comment_text'].value_counts()

Well said.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                196
No.                                                       

In [26]:
df.shape

(2125743, 2)

In [27]:
df['dups']=df.groupby('comment_text').transform('cumcount')

In [28]:
df['dups'].unique()

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

In [29]:
df.loc[df['dups']>0,:]

,comment_text,target,dups
223633,"Tuesday Jan. 13, members of \nthis socalled 'm...",1,1
223838,You're an idiot.,1,1
224160,While stumping for the bond money the powers t...,0,1
224217,I have eaten at the Imperial and enjoyed the e...,0,1
224492,No. Not legal under OLCC regulations! See und...,0,1
...,...,...,...
2125302,"Well-trolled, indeed. As far as Civil Comments...",0,5
2125338,test,0,75
2125384,What?,0,85
2125420,What?,0,86


In [30]:
df[df['comment_text']==str(df.loc[223838,'comment_text'])]

,comment_text,target,dups
204865,You're an idiot.,1,0
223838,You're an idiot.,1,1
256830,You're an idiot.,1,2
312554,You're an idiot.,1,3
339483,You're an idiot.,1,4
364426,You're an idiot.,1,5
429809,You're an idiot.,1,6
503417,You're an idiot.,1,7
586973,You're an idiot.,1,8
759852,You're an idiot.,1,9


In [31]:
df=df[df['dups']==0].drop('dups',axis=1).reset_index(drop=True)

In [32]:
df.shape

(2099954, 2)

In [33]:
df.head()

,comment_text,target
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [34]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense,BatchNormalization,Input,Dropout,ReLU
import tensorflow as tf

In [35]:
test_data=pd.read_csv('../input/jigsaw-multilingual-toxic-comment-classification/validation.csv')

In [36]:
test_data.columns=['id','comment_text','lang','target']

In [37]:
del df1,df2
import gc
gc.collect()

225

In [38]:
test_data.head()

,id,comment_text,lang,target
0,0,Este usuario ni siquiera llega al rango de ...,es,0
1,1,Il testo di questa voce pare esser scopiazzato...,it,0
2,2,Vale. Sólo expongo mi pasado. Todo tiempo pasa...,es,1
3,3,Bu maddenin alt başlığı olarak uluslararası i...,tr,0
4,4,Belçika nın şehirlerinin yanında ilçe ve belde...,tr,0


xlmroberta

In [39]:
sent_length=210

In [40]:
!pip install sentencepiece

In [41]:
x_tokenizer=XLMRobertaTokenizer.from_pretrained('jplu/tf-xlm-roberta-base')

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

In [42]:
x_tokenizer.encode_plus('kaddddc','adfca',max_length=20,padding='max_length',truncation='longest_first')

{'input_ids': [0, 1072, 4028, 71, 238, 2, 2, 606, 420, 408, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]}

In [43]:
x_tokenizer.decode(x_tokenizer.encode_plus('kaddddc','adfca',max_length=20,padding='max_length',truncation='longest_first').get('input_ids'))

'<s> kaddddc</s></s> adfca</s><pad><pad><pad><pad><pad><pad><pad><pad><pad>'

In [44]:
train_data=df.copy()

In [45]:
model_inputs={'input_ids':np.empty((train_data.shape[0],sent_length),np.int32),
              'attention_mask':np.empty((train_data.shape[0],sent_length),np.int32)}
model_outputs=train_data['target'].values
for i in tqdm(range(train_data.shape[0])):
  tokenized=x_tokenizer.tokenize(str(train_data.loc[i,'comment_text']))
  if len(tokenized)>sent_length-2:
    tokenized=['<s>']+tokenized[:(sent_length-2)]+['</s>']
  else:
    curr_len=len(tokenized)
    tokenized=['<s>']+tokenized+['</s>']+['<pad>']*(sent_length-curr_len-2)
  token_ids=x_tokenizer.convert_tokens_to_ids(tokenized)
  mask=np.char.not_equal('<pad>',tokenized).astype(np.int32)
  model_inputs['input_ids'][i,]=token_ids
  model_inputs['attention_mask'][i,]=mask
################################################################################################
valid_inputs={'input_ids':np.empty((test_data.shape[0],sent_length),np.int32),
              'attention_mask':np.empty((test_data.shape[0],sent_length),np.int32)}
valid_outputs=test_data['target'].values
for i in tqdm(range(test_data.shape[0])):
  tokenized=x_tokenizer.tokenize(str(test_data.loc[i,'comment_text']))
  if len(tokenized)>sent_length-2:
    tokenized=['<s>']+tokenized[:(sent_length-2)]+['</s>']
  else:
    curr_len=len(tokenized)
    tokenized=['<s>']+tokenized+['</s>']+['<pad>']*(sent_length-curr_len-2)
  token_ids=x_tokenizer.convert_tokens_to_ids(tokenized)
  mask=np.char.not_equal('<pad>',tokenized).astype(np.int32)
  valid_inputs['input_ids'][i,]=token_ids
  valid_inputs['attention_mask'][i,]=mask

100%|██████████| 8000/8000 [00:06<00:00, 1319.25it/s]


In [46]:
del df
import gc
gc.collect()

74

In [47]:
import pickle
with open('train_data_xlm_roberta.pkl','wb') as f:
  pickle.dump(model_inputs,f)
with open('train_data_labels_xlm_roberta.pkl','wb') as f:
  pickle.dump(model_outputs,f)
with open('valid_data_xlm_roberta.pkl','wb') as f:
  pickle.dump(valid_inputs,f)
with open('valid_data_labels_xlm_roberta.pkl','wb') as f:
  pickle.dump(valid_outputs,f)